# COGS 118A - Project Checkpoint

# Names

- Guanyu Zhu
- Tze Howe Yap
- David Chen
- Cassidy Lu

# Abstract 

Airbnb is a well known internet marketplace for short-term home and apartment rentals. Hosts often face difficulty in determining an optimal price for their listings. Both overpricing and underpricing can lead to revenue loss. To solve this problem, we use previews listed Airbnb in San Diego including their price and descriptions to encode those into machine learning models. 
 We aim to develop a model that can successfully predict Airbnb prices in San Diego based on relevant property features. We measure the model’s accuracy using R^2 and calculate the loss using RMSE. 

# Background

On May 5, 2023, the CDC announced the End of the Federal COVID-19 Public Health Emergency (PHE) Declaration, signaling an official end to this unprecedented pandemic. What's to come is the recovery of the global travel market. Across the entire world, there are currently over 4 million+ Airbnb hosts worldwide and 6 million+ active listings on the platform. In addition to the staggering number of hosts, there are more than 150 million users, and the overall number of bookings through Airbnb has surpassed 100 million.<a name="airbnbstats"></a>[<sup>[1]</sup>](#airbnbstats) In the area of San Diego alone, there are a total of 12,871 listings, with 87.1% of them being short-term rentals. Additionally, 66.3% of hosts have multiple listings, indicating that most hosts have more than one property on the market.<a name="airbnbsd"></a>[<sup>[2]</sup>](#airbnbsd) For those with a large number of listing properties, it becomes challenging to thoroughly investigate and change prices daily. Therefore, having a comparable prediction as a reference for their decision would be beneficial. The potential of the short-term rental market is enormous and growing exponentially. However, it's not just people with multiple listings who struggle with setting a fair price; new hosts also typically face difficulties when initially deciding on the price for their listings.

When you try to list your home on airbnb, you’ll need to do market research on the similar homes near you, finding out the average price of the market, and then determine the price that you would like to set for your home. The problem is that you can’t list a price and never change it forever. You need to change the price frequently based on market price, which means each time you want to change the price you need to do the market research again, which is a lot of work. In this case, if you have a program that can predict the price base on the model that has previously been trained on similar data from already existing listings would be pretty helpful. 


# Problem Statement

Setting a reasonable price for an airbnb host is hard to achieve. They need to predict other similar listings’ prices in order to maximize the profit. We aim to utilize machine learning algorithms to analyze past data and train a predictive model that can most accurately predict the most profitable airbnb price based on the property’s features and location. 


# Data

UPDATED FROM PROPOSAL!

You should have obtained and cleaned (if necessary) data you will use for this project.

Please give the following infomration for each dataset you are using
- link/reference to obtain it
- description of the size of the dataset (# of variables, # of observations)
- what an observation consists of
- what some critical variables are, how they are represented
- any special handling, transformations, cleaning, etc you have done should be demonstrated here!


In [1]:
# imports 
import numpy as np
import pandas as pd

import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
# load in both dataframe
raw_listing = pd.read_csv('data/listings.csv')
raw_calendar = pd.read_csv('data/calendar.csv.gz')

## Data Cleaning

### Listing 

In [3]:
# Get useful column assign to new dataframe
featured_listing = raw_listing[['id', 'neighbourhood', 'latitude', 'longitude', 
                                'room_type']]

In [4]:
featured_listing.head()

,id,neighbourhood,latitude,longitude,room_type
0,484515,Hollywood,25.993770,-80.118300,Entire home/apt
1,827736378366911479,Fort Lauderdale,26.093936,-80.137591,Entire home/apt
2,592589963829194972,Pompano Beach,26.223850,-80.090990,Private room
3,45990221,Pompano Beach,26.226130,-80.158590,Entire home/apt
4,11972837,Pompano Beach,26.226220,-80.158710,Entire home/apt


In [6]:
# converted column data types from the original DataFrame as we saw fit

featured_listing['neighbourhood'] = featured_listing['neighbourhood'].astype('string')
featured_listing['room_type'] = featured_listing['room_type'].astype('string')
featured_listing.dtypes

id                 int64
neighbourhood     string
latitude         float64
longitude        float64
room_type         string
dtype: object

### Calendar 

In [7]:
# investigate two cloumns 'price' and 'adjusted_price'
raw_calendar[(raw_calendar['price'] != raw_calendar['adjusted_price'])].head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
730,129099,2023-03-28,t,$250.00,$225.00,28.0,365.0
731,129099,2023-03-29,t,$250.00,$225.00,28.0,365.0
732,129099,2023-03-30,t,$250.00,$225.00,28.0,365.0
733,129099,2023-03-31,t,$250.00,$225.00,28.0,365.0
734,129099,2023-04-01,t,$250.00,$225.00,28.0,365.0


In [8]:
# Get useful columns 
featured_calendar = raw_calendar[['listing_id', 'date', 'available', 'adjusted_price', 
                                  'minimum_nights', 'maximum_nights']]

In [9]:
featured_calendar.head()

,listing_id,date,available,adjusted_price,minimum_nights,maximum_nights
0,69824,2023-03-28,f,$150.00,30.0,365.0
1,69824,2023-03-29,f,$150.00,30.0,365.0
2,69824,2023-03-30,f,$150.00,30.0,365.0
3,69824,2023-03-31,t,$150.00,30.0,365.0
4,69824,2023-04-01,t,$150.00,30.0,365.0


In [11]:
# Helper function to change columns type
def clean_price(price):
    return price.strip('$').replace(',', '')

def to_boolean(available):
    if available == 't':
        return True
    else:
        return False

In [12]:
# converted column data types from the original DataFrame as we saw fit 

featured_calendar['adjusted_price'] = featured_calendar['adjusted_price'].astype(str).apply(clean_price).astype(float)
featured_calendar['date'] = pd.to_datetime(featured_calendar['date'])
featured_calendar['available'] = featured_calendar['available'].astype(str).apply(to_boolean)

In [13]:
# get only the row where available equals to False
featured_stay_calendar = featured_calendar[featured_calendar['available'] == False]
featured_stay_calendar.head()

,listing_id,date,available,adjusted_price,minimum_nights,maximum_nights
0,69824,2023-03-28,False,150.0,30.0,365.0
1,69824,2023-03-29,False,150.0,30.0,365.0
2,69824,2023-03-30,False,150.0,30.0,365.0
18,69824,2023-04-15,False,150.0,30.0,365.0
19,69824,2023-04-16,False,150.0,30.0,365.0


In [14]:
# Merge two dataframe
merged_df = featured_listing.merge(featured_stay_calendar, left_on = 'id', right_on = 'listing_id')
merged_df.head()

,id,neighbourhood,latitude,longitude,room_type,listing_id,date,available,adjusted_price,minimum_nights,maximum_nights
0,484515,Hollywood,25.993770,-80.118300,Entire home/apt,484515,2023-03-28,False,297.0,7.0,1125.0
1,827736378366911479,Fort Lauderdale,26.093936,-80.137591,Entire home/apt,827736378366911479,2023-03-27,False,222.0,1.0,365.0
2,827736378366911479,Fort Lauderdale,26.093936,-80.137591,Entire home/apt,827736378366911479,2023-03-28,False,222.0,1.0,365.0
3,827736378366911479,Fort Lauderdale,26.093936,-80.137591,Entire home/apt,827736378366911479,2023-03-29,False,222.0,1.0,365.0
4,827736378366911479,Fort Lauderdale,26.093936,-80.137591,Entire home/apt,827736378366911479,2023-03-30,False,222.0,1.0,365.0


In [15]:
# Assess missing data
merged_df.isna().any()

id                False
neighbourhood     False
latitude          False
longitude         False
room_type         False
listing_id        False
date              False
available         False
adjusted_price     True
minimum_nights     True
maximum_nights     True
dtype: bool

In [17]:
# Drop the row if adjusted price is missing 
merged_df.dropna(subset=['adjusted_price'])
merged_df.head()

,id,neighbourhood,latitude,longitude,room_type,listing_id,date,available,adjusted_price,minimum_nights,maximum_nights
0,484515,Hollywood,25.993770,-80.118300,Entire home/apt,484515,2023-03-28,False,297.0,7.0,1125.0
1,827736378366911479,Fort Lauderdale,26.093936,-80.137591,Entire home/apt,827736378366911479,2023-03-27,False,222.0,1.0,365.0
2,827736378366911479,Fort Lauderdale,26.093936,-80.137591,Entire home/apt,827736378366911479,2023-03-28,False,222.0,1.0,365.0
3,827736378366911479,Fort Lauderdale,26.093936,-80.137591,Entire home/apt,827736378366911479,2023-03-29,False,222.0,1.0,365.0
4,827736378366911479,Fort Lauderdale,26.093936,-80.137591,Entire home/apt,827736378366911479,2023-03-30,False,222.0,1.0,365.0


1. clean both df -> change the column type to the correct type (done)
2. try merging the dfs base on the id in featured_listing df (done)
3. acssess missing rows (maybe drop them) (done)
    clean features 
4. Univariate Analysis -> graph (histgram)
5. Bivariate Analysis -> graph 

5. extract feature 
6. create basic pipline -> build base model 

# Proposed Solution

We plan to build at least two models using Sklearn modules to compare their results for predicting airbnb prices in San Diego. Our thought is to use RandomForestRegressor model and polynomial regression model. Since our data consist of a decent amount of categorical variables as well as numerical variables, we will use one-hot encoding before putting all those data into two models. In order to test our model and to see how our models can generalize on unseen data, we will split our data into testing and training. For the training data, we will then use k-fold validation and GridSearchCV to make sure our data is not overfitting and tune our hyperparameter accordingly. 
Our General Plan as for now consist of those following step:
1. Data cleaning and using python and pandas
    - Drop out irrelevant features and data
    - Univariate Analysis
    - Bivariate Analysis
    - Assessment of Missingness 
2. Feature extraction using python and regex 
    - One-hot encoding for categorical features 
    - Assess numerical features and standardize relevant features 
3. Split the dataset into testing and training 
4. Build a baseline model 
5. Implement two models, use k-fold cross validation and GridSearchCV
6. After achieving good prediction use the best performed model on testing dataset

# Evaluation Metrics

An appropriate evaluation metric, such as Root Mean Square Error (RMSE) will be used to quantify the performance of the model. This metric quantifies the difference between predicted and actual prices, providing an indication of the model's accuracy. The Root Mean Square Error is calculated by the following formula:
$$
\text{RMSE}(y, \hat{y}) = \sqrt{\frac{\sum_{i=0}^{N - 1} (y_i - \hat{y}_i)^2}{N}}
$$
where N is the number of the data samples, is the ith measurement, and  is the corresponding predictions of the sample.

# Preliminary results

NEW SECTION!

Please show any preliminary results you have managed to obtain.

Examples would include:
- Analyzing the suitability of a dataset or alogrithm for prediction/solving your problem 
- Performing feature selection or hand-designing features from the raw data. Describe the features available/created and/or show the code for selection/creation
- Showing the performance of a base model/hyper-parameter setting.  Solve the task with one "default" algorithm and characterize the performance level of that base model.
- Learning curves or validation curves for a particular model
- Tables/graphs showing the performance of different models/hyper-parameters



# Ethics & Privacy

- Data Collection:
    - Informed consent:
Our dataset is extracted from official documentation published by Airbnb. The information can be searched online and hosts sign a form of consent when the properties are listed on the market, so there should not be any legal issues.
    - Collection bias:
The dataset is comprehensively collected from every single listing in the area of San Diego. So, there is no foreseeable bias in the process of data collection.
    - Limit PII exposure:
The dataset contains names of the hosts, so we will delete the sensitive information such as their ID and names of the hosts in the data cleaning processes.


- Data Storage
    - Data Security and Right to be forgotten
In compliance with regulations such as the GDPR, we respect the 'right to be forgotten.' If an individual whose data is included in our project requests for their data to be erased, we are obligated to do so, provided there are no legal grounds for retaining it. This helps to ensure that individuals maintain control over their personal data.
    - Data retention plan:
We recognize the importance of not retaining data longer than necessary. Our default position will be to retain data for only as long as it is required for the purpose of the project. Once the data is no longer necessary, we will safely and securely delete it.


- Analysis
    - Missing perspectives
For this project, we do not obtain the source of expertise knowledge or any affected communities. 
    - Dataset bias: 
Our datasets record the fundamental features of listings in San Diego are. The data it contained does not contain any information that can be related to stereotype perpetuation or imbalance classes. One variable that might be confounding in our dataset is “license”. According to our research, Airbnb's official forum provides information regarding this feature. Excerpting from Airbnb’s official response: “Some cities require Hosts to obtain a license or registration number to list their places on Airbnb. The corresponding field on their listing allows them to show that number for compliance.” This provides us an understanding of what a license is and how it applies to the property listing. Additionally, Airbnb also suggests that “license” might not be necessary for some cities, depending on the policies. For those properties that have status ‘city registration pending’ or ‘exempt’, Airbnb claims that it will not affect the veracity and safety of the listings.
    - Honest Representation:
Our visualizations, summary statistics, and reports will honestly represent the underlying data.
    - Privacy in Analysis:
Yes, we will remove the data that are related to IDs or host name in the data cleaning process.
    - Auditability:
Yes, the process will be able to reproduce using the dataset we provides and the technique we used. 



- Modeling
    - Proxy Discrimination:
We will ensure that the model does not inadvertently result in discrimination through the use of proxy variables - variables that correlate with protected characteristics such as race, age, or gender.
    - Fairness across groups:
We will prioritize fairness across all groups to ensure that no one group will receive favor based on any characteristics. We will utilize techniques to check if our model is biased, and if it is, we will mitigate it and re-evaluate our model.
    - Metric selection:
RMSE
    - Explainability:
We aim to choose a model that achieves a balance between explainability and accuracy. While more complex models offer better accuracy and predictiveness, it will more often be harder to explain. It is important for us to understand how a model is making its decisions.


- Deployment
    - Monitoring and evaluation:
Once the model is deployed, there is a need to monitor the performance and fairness continuously. We will track metrics like the model’s accuracy and potential bias over time and across different groups.
Redress
    - Roll back:
A roll-back plan will be in place to revert to a previous version of the model or even to stop using the model altogether if serious issues arise. This could be necessary if the model starts performing poorly or if significant bias is discovered that can't be quickly mitigated. 
    - Redress:
We will officially apologize to those who are harmed by our project and provide contact information for them to reach out for a solution
    - Unintended use:
Some unintended use for our project can be customers/guests using our results to argue with the host for lower prices, which might have negative effects on the revenue of listing properties.


# Team Expectations 

Put things here that cement how you will interact/communicate as a team, how you will handle conflict and difficulty, how you will handle making decisions and setting goals/schedule, how much work you expect from each other, how you will handle deadlines, etc...
* *All team members actively engage in the discussion of our project*
* *All team members have completed their respective tasks on time while maintaining a high standard of quality in their work *
* *All team members contribute equally to solution and information research*

# Project Timeline Proposal

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 5/10  |  7-8 PM |  Brainstorm topics/questions (all)  | Determine best form of communication; Discuss and decide on final project topic; discuss hypothesis; begin background research | 
| 5/15  |  5-8 PM |  Do background research on topic (all) | Discuss ideal dataset(s) and ethics; draft project proposal | 
| 5/17  | 4-8 PM  | Edit, finalize, and submit proposal; Search for datasets (all)  | Discuss Wrangling and possible analytical approaches; Assign group members to lead each specific part   |


# Footnotes
<a name="airbnbstats"></a>1.[^](#airbnbstats): “Airbnb Statistics [2023]: User & Market Growth Data.” SearchLogistics, 28 Apr. 2023, www.searchlogistics.com/learn/statistics/airbnb-statistics/. <br>
<a name="airbnbsd"></a>2.[^](#airbnbsd): “San Diego.” Inside Airbnb: Home, insideairbnb.com/san-diego. Accessed 17 May 2023. 
